## Import modules

In [1]:
import pickle
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorchtools
import glob as gl
import random
import os
import time
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import make_regression
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import log_loss
from numpy import hstack
from numpy import vstack

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

## Data loading

In [2]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()
find_directory(back_num = 1, foldername = ['Dataset'], filename = 'bat_dict.pkl')

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\All dataset'

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\Dataset\\bat_dict.pkl'

In [3]:
bat_dict_add_1_2 = find_directory(back_num = 2, foldername = ['Dataset'], filename = 'bat_1_2_dict_VITQcQd.pkl')
with open(bat_dict_add_1_2, 'rb') as tf:
    bat_dict_1_2 = pickle.load(tf)

bat_sel_dict = bat_dict_1_2
max_time = 120

In [4]:
# Tensor dimension: (cell, cycle, time, variable)
num_cells = len(bat_sel_dict.keys())
regression_cycles = [45, 65, 85, 100]

# variable = [Qc, I, V, T, Qd]
num_variables = len(bat_sel_dict['b1c0']['1']['1.0'].keys())

for reg in regression_cycles:
    #'''RNN 가변길이 다룰수 있게?? 고민필요'''
    globals()[f'x_tensor_{reg}'] = torch.zeros(num_cells, reg, max_time, num_variables)
    cell_index = 1
    for cell in bat_sel_dict.keys():
        for cycle in range(1, reg+1):
            for time in bat_sel_dict[cell][str(cycle)].keys():
    #             print(bat_sel_dict[cell][str(cycle)][str(time)])
                value_list = [i for i in bat_sel_dict[cell][str(cycle)][str(time)].values()]
    #             print(value_list)
                globals()[f'x_tensor_{reg}'][cell_index-1, cycle-1, int(float(time)*max_time/60), :] = torch.FloatTensor(value_list)

        cell_index += 1

    globals()[f'x_tensor_{reg}'].size()

num_variables = x_tensor_100.shape[3]
time_length = x_tensor_100.shape[2]
# (cycle_index-1, cycle-1, time-1, variable(Qc, I, V, T, Qd))
# x_tensor[123, 99, 30, :]

torch.Size([124, 45, 120, 5])

torch.Size([124, 65, 120, 5])

torch.Size([124, 85, 120, 5])

torch.Size([124, 100, 120, 5])

In [5]:
y_df_add = find_directory(back_num = 2, foldername = ['Dataset', 'y_df.csv'])
y_df = pd.read_csv(y_df_add)

# 2. Regression problem
# KO, KP, CL
y_cl_numpy = y_df['Cycle life'].to_numpy()
y_kp_numpy = y_df['kneepoints'].to_numpy()
y_ko_numpy = y_df['kneeonsets'].to_numpy()
# y_ko_numpy.shape

y_regression_numpy = y_ko_numpy

y_ko_tensor = torch.FloatTensor(y_ko_numpy)
y_ko_tensor.size()

torch.Size([124])

In [6]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed) # torch 
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True # cudnn
    torch.backends.cudnn.benchmark = False # cudnn
    np.random.seed(random_seed) # numpy
    random.seed(random_seed) # random

In [7]:
x_tensor_100.shape

torch.Size([124, 100, 120, 5])

### Dataloading as 2D CNN's method

In [8]:
# y: no scaling
class dataPrep_RNN_CNN(Dataset):  
    def __init__(self, x_tensor, y_tensor, batch_size, scaler, purpose : str, test_split: float):
        setRandomSeed(random_seed = 100)
        self.xdata = torch.permute(x_tensor, (0, 3, 2, 1))
        self.ydata = y_tensor
        
        self.batch_size = batch_size
                
        self.xscaler = scaler()
        self.purpose = purpose
        
        self.test_split = test_split
        
        self.train_size, self.val_size, self.test_size = self.get_splits()
        
        # Tensor dimension: (cell, variable, time, cycle)
        # If problem emerges, check this part first.
        for i in range(len(self.xdata)):
            for j in range(self.xdata.shape[1]):
                temp = np.expand_dims(self.xscaler.fit_transform(self.xdata[i,j,:,:]), axis=0)
                if j ==0:
                    temp2 = temp
                else:
                    temp2 = np.vstack((temp2,temp))
            temp2 = np.expand_dims(temp2, axis=0)
            if i==0:
                self.xdata_scaled = temp2
            else:
                self.xdata_scaled = np.vstack((self.xdata_scaled, temp2))

        self.ydata_scaled = self.ydata
        
        self.x_train_scaled, self.x_test_scaled, self.y_train_scaled, self.y_test_scaled = train_test_split(self.xdata_scaled, 
                                                                                                            self.ydata_scaled, 
                                                                                                            test_size = self.test_size, 
                                                                                                            shuffle = True, 
                                                                                                            random_state=100)
        
        self.x_train_scaled, self.x_val_scaled, self.y_train_scaled, self.y_val_scaled = train_test_split(self.x_train_scaled,
                                                                                                         self.y_train_scaled,
                                                                                                         test_size = self.val_size, 
                                                                                                         shuffle = True, 
                                                                                                         random_state = 100)
        
        print(self.x_train_scaled.shape, self.x_val_scaled.shape, self.x_test_scaled.shape,
              self.y_train_scaled.shape, self.y_val_scaled.shape, self.y_test_scaled.shape)
        
        self.train = list(zip(self.x_train_scaled, self.y_train_scaled))
        self.val = list(zip(self.x_val_scaled, self.y_val_scaled))
        self.test = list(zip(self.x_test_scaled, self.y_test_scaled))
                
        self.train_dataloader = DataLoader(self.train, batch_size = self.batch_size, shuffle=True)
        self.val_dataloader = DataLoader(self.val, batch_size = self.batch_size, shuffle=True)
        self.test_dataloader = DataLoader(self.test, batch_size = self.batch_size, shuffle=True)
            
    # split the datasets into training and testing
    def get_splits(self):
        # test size
        test_size = int(self.test_split*len(self.xdata))
        # val size
        val_size = int(0.2*(len(self.xdata) - test_size))
        # train_size
        train_size = len(self.xdata) - val_size - test_size
        return train_size, val_size, test_size
    
    # Returns dataloaders for training and validation datasets
    def train_data(self):
        return self.train_dataloader, self.val_dataloader
    
    # Returns dataloader for test datasets
    def test_data(self):
        return self.test_dataloader
    
    def scaler(self, purpose):
#         if purpose != 'classification':
#             return self.xscaler, self.yscaler
#         else:
        return self.xscaler
    
    # finding length of x 
    def __len__(self):
        return len(self.X)
    
    # indexing rows for calling
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]


In [9]:
# reg_dataset = dataPrep1D(x_tensor, y_ko_tensor, batch_size = 16, scaler = MinMaxScaler, purpose = 'regression', test_split = 0.2)
# reg_train_dataloader, reg_val_dataloader = reg_dataset.train_data()
# reg_test_dataloader = reg_dataset.test_data()
# reg_xscaler = reg_dataset.scaler('regression')

regression_cycles = [45, 65, 85, 100]
# regression_cycles = [100]


# x와 y가 1대1 대응이어야 dataloader를 만들 수 있어서 2D array 그대로 놓고 학습시 tensor transform 예정
for reg in regression_cycles:
    globals()[f'reg_dataset_{reg}'] = dataPrep_RNN_CNN(globals()[f'x_tensor_{reg}'], y_ko_tensor, batch_size = 4, scaler = MinMaxScaler, purpose =  'regression', test_split = 0.2)
    globals()[f'reg_train_dataloader_{reg}'], globals()[f'reg_val_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].train_data()
    globals()[f'reg_test_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].test_data()
    globals()[f'reg_xscaler_{reg}'] = globals()[f'reg_dataset_{reg}'].scaler('regression')

(80, 5, 120, 45) (20, 5, 120, 45) (24, 5, 120, 45) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 65) (20, 5, 120, 65) (24, 5, 120, 65) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 85) (20, 5, 120, 85) (24, 5, 120, 85) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 100) (20, 5, 120, 100) (24, 5, 120, 100) torch.Size([80]) torch.Size([20]) torch.Size([24])


In [10]:
# train_features, train_labels = next(iter(cla_train_dataloader))
# print(f"Classification case_Inputs batch shape(Batch size, num_vars, timestep, cycles): {train_features.size()}")
# print(f"Classification case_Outputs batch shape(Batch size): {train_labels.size()}")

reg = 100
train_features, train_labels = next(iter(globals()[f'reg_train_dataloader_{reg}']))
print(f'reg = {reg}')
print(f"Regression case_Inputs batch shape(Batch size, num_vars, timestep, cycles): {train_features.size()}")
print(f"Regression case_Outputs batch shape(Batch size): {train_labels.size()}")

train_features.float().type()

reg = 100
Regression case_Inputs batch shape(Batch size, num_vars, timestep, cycles): torch.Size([4, 5, 120, 100])
Regression case_Outputs batch shape(Batch size): torch.Size([4])


'torch.FloatTensor'

## Define Model Class 

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [12]:
class RNNcell(nn.Module):
    def __init__(self, model, input_size, hidden_size, dropout = 0.1, bidirectional = False, num_layer=1):
        super(RNNcell, self).__init__()
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
            
        self.model = model
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.D = 1 + self.bidirectional
        self.batch_size = 4
        
        #Batch size * cycles, timestep, input_size
        if self.model.endswith('RNN'):
            self.rnn = nn.RNN(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('LSTM'):
            self.rnn = nn.LSTM(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('GRU'):
            self.rnn = nn.GRU(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
            
        for name, param in self.rnn.named_parameters():
            if name.startswith('weight'):
                nn.init.xavier_uniform(param)
            else:
                nn.init.normal(param)
        
    def forward(self, x):
        # x made from (cell * n_cy, timestep(seq_len), num_vars)
        self.batch_size = x.size(0)
        
        D = 1 + self.bidirectional
        #out: Containing output features h_t from the last layer of LSTM for each t(if bidirectional ==True: concat of forward and backward hidden states)
        #h_n: Containing final hidden state in the sequence
        h0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
        if self.model.endswith('LSTM'):
            c0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
            # c_n: Containing final cell state in the sequence
            out, (hn, cn) = self.rnn(x, (h0, c0))
        else:
            out, hn = self.rnn(x, h0.detach())
        return out, hn

In [13]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(SelfAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads
        
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, inputs):
        # inputs shape: [batch_size, seq_len, hidden_size]
        batch_size, seq_len, hidden_size = inputs.size()
        
        # Compute query, key, and value matrices
        query = self.query(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        key = self.key(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        value = self.value(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        
        # Compute scaled dot-product attention scores
        # attention_scores: [batch_size, num_heads, seq_len, seq_len]
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_size, dtype=torch.float32))
        attention_probs = self.softmax(attention_scores)
        
        # Apply attention to the value matrix and concatenate heads
        # context: [batch_size, num_heads, seq_len, head_size]=>[batch_size, seq_len, num_heads, head_size] 
        # => [batch_size, seq_len, num_heads * head_size]
        context = torch.matmul(attention_probs, value).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
#         context = nn.Linear(hidden_size, 1)(context)
        return attention_probs, context

In [14]:
class RNN_CA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, nhead2, fil2, pool2, npool2, fsize2, psize2, mids, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        super(RNN_CA_1DCNN, self).__init__()
        setRandomSeed()
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        # Pooling/Nonpooling layers
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # Params for self-attention(ca)
        self.nh2 = nhead2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
#         self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.ca = SelfAttention(self.D1*hid1, self.nh2)
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(hn1.size(1)/self.num_cycles)

        # Reshape for input
        # out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)
        # outs : (batch_size(num_cell), num_cycle, timestep, D1*hid1)
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        # outs : (batch_size(num_cell), D1*hid1, num_cycle, timestep)
        outs1 = torch.permute(outs1, (0, 3, 1, 2))
        # ta: (batch_size, num_cycle, timestep, 1(D1*hid1->1))
#         ta = self.sm1(self.lin1(outs1))
        # ta_outs: (batch_size, num_cycle, timestep, D1*hid1)=>(batch_size, D1*hid1, num_cycle, timestep)
#         ta_outs = torch.permute(ta*outs1, (0, 3, 1, 2))
        # ct_vec: (batch_size, D1*hid1, num_cycle)
#         ct_vec = torch.sum(ta_outs, -1)
        # ct_vec: (batch_size, D1*hid1, num_cycle)
        ct_vec = outs1[:, :, :, -1].squeeze()
        # ct_vec shape: [batch_size, num_cycle, D1*hid1(hidden_size)]
        # hidden_size = num_heads * head_size
        # ct_vec transform back: [batch_size, num_heads, num_cycle, head_size]=>[batch_size, num_cycle, num_heads, head_size] 
        # => [batch_size, num_cycle, num_heads * head_size(hidden_size)]
        # ca: matmul(query[batch_size, num_heads, num_cycle, head_size], key^T[batch_size, num_heads, head_size, num_cycle])
        # ca shape: [batch_size, num_heads, num_cycle, num_cycle]
        ca, ct_vec = self.ca(ct_vec.transpose(2, 1))
#         print(f"After self attention: ca.size: {ca.size()}, ct_vec.size: {ct_vec.size()}")
        # ct_vec shape: [batch_size, D1*hid1, num_cycle]
        ct_vec = ct_vec.transpose(2, 1)
        
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        # ct_vec from final convolutional layer: (batch_size, self.in2*self.Lout)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        # final_out: (batch_size, 1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ca

In [15]:
# Function for saving and loading of training history
def save_data(D3_array, filename):
    with open(filename,"wb") as dat_:
        pickle.dump(D3_array,dat_)
        
def load_data(filename):
    with open(filename,"rb") as ld:
        x_temp = pickle.load(ld)
    return x_temp

# Early stopping class
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.best_epoch = 1
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, epoch, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_epoch = epoch
            self.best_model = self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'At epoch {self.best_epoch}: Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
        return model

In [16]:
class train:
    def __init__(self, model: nn.Module, train_dataloader, val_dataloader, test_dataloader, epoch: int, learning_rate=0.01, patience = 5, verbose = False):
        super().__init__()
        #   Reprocudtion
        setRandomSeed(100)
        random_seed = 100
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(random_seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        os.environ['PYTHONHASHSEED'] = str(random_seed)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)

        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.test_dataloader = test_dataloader
        self.epoch = epoch
        self.learning_rate = learning_rate
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.learning_rate)
        self.early_stopping = EarlyStopping(patience = patience, verbose = verbose)

        self.criterion = nn.MSELoss()
        self.name = ['train_rmse', 'val_rmse', 'test_rmse']
        
    def EvalModel(self):
        EvalPredictions, EvalActuals = list(), list()
        TestPredictions, TestActuals = list(), list()
        self.model.eval()
        with torch.no_grad():
            for n, (inputs, targets) in enumerate(self.val_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                EvalPredictions.append(yhat)
                EvalActuals.append(actual)

            EvalPredictions, EvalActuals = vstack(EvalPredictions), vstack(EvalActuals)
            val_rmse = mean_squared_error(EvalActuals, EvalPredictions, squared = False)
            
            for n, (inputs, targets) in enumerate(self.test_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                TestPredictions.append(yhat)
                TestActuals.append(actual)

            TestPredictions, TestActuals = vstack(TestPredictions), vstack(TestActuals)
            test_rmse = mean_squared_error(TestActuals, TestPredictions, squared = False)
            
            return val_rmse, test_rmse

    def TrainModel(self):
        loss_history = []
        self.model.train()
        for i in range(self.epoch):
            predictions, actuals = list(), list()
            for n, (inputs, targets) in enumerate(self.train_dataloader):
                self.model.train()
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                self.optimizer.zero_grad()

                yhat, ca = self.model(inputs)
                    
                loss = self.criterion(yhat, targets)
                loss.backward()
                self.optimizer.step()
                
                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')
                    
                yhat = yhat.detach().numpy()
                predictions.append(yhat)
                
                targets = targets.numpy()
                actuals.append(targets)

            # evaluation of entire train data
            predictions, actuals = vstack(predictions), vstack(actuals)
            
            train_rmse = mean_squared_error(actuals, predictions, squared = False)

            # evaluation on validation data
            val_rmse, test_rmse = self.EvalModel()
            early_factor = val_rmse
            
            # training loss history
            loss_history.append([train_rmse, val_rmse, test_rmse])

            self.early_stopping(i+1, early_factor, self.model)
            
            if self.early_stopping.early_stop:
                print("Early stopping at best epoch: ", self.early_stopping.best_epoch)
                self.best_loss = loss_history[self.early_stopping.best_epoch-1]
                loss_history = loss_history[:self.early_stopping.best_epoch]
                
                return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ca
            
        print(f"Ends at final epoch {self.epoch}")
        print(f"Best epoch: {self.early_stopping.best_epoch}")
        self.best_loss = loss_history[self.early_stopping.best_epoch-1]
        loss_history = loss_history[:self.early_stopping.best_epoch]
        
        return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ca
        
    def predict(self, data: torch.Tensor):
        return self.model(data)

## Implementation

In [17]:
def history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ca_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ca'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ca_add

In [19]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [1, 5]

import time

for [n_ep, patience] in ep_pats:
    print(f"epoch = {n_ep}, patience = {patience}")
    trdl = globals()[f'reg_train_dataloader_{num_cycles}']
    vdl = globals()[f'reg_val_dataloader_{num_cycles}']
    tedl = globals()[f'reg_test_dataloader_{num_cycles}']
    for rnn1, hid1 in itertools.product(rnns, hids):
        bi1 = False
        if rnn1.startswith('Bi'):
            bi1 = True
        for pool2, npool2 in itertools.product(pools, npools):
            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
            for fil2, lr in itertools.product(num_fils, lrs):
                for nh2 in nh2s:
                    print(f"rnn = {rnn1}, bi = {bi1}, hid = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                    all_hid1 = hid1 * nh2

                    start = time.time()
                    # Construct CNN
                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                    # Train
                    model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                    best_model, best_loss, history, ca = model_train.TrainModel()

                    print('best_loss = '+ str(best_loss))

                    history_add, state_dict_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                    # saving best_epoch, loss history, ta score
                    save_data(history, history_add)
                    save_data(best_model.state_dict(), state_dict_add)
                    save_data(ca, ca_add)

                    print("time: ", time.time()-start)
                    print("\n\n")

epoch = 1000, patience = 10
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid = 3, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  153
best_loss = [291.35364, 246.916, 266.5745]
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_1000_10\RNN_CA_1DCNN\LSTM_1D CNN\train_history created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_1000_10\RNN_CA_1DCNN\LSTM_1D CNN\model created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_1000_10\RNN_CA_1DCNN\LSTM_1D CNN\ca created
time:  27.769400596618652



rnn = LSTM, bi = False, hid = 3, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [571.1423, 542.6277, 527.1941]
time:  2.3689987659454346



rnn = LSTM, bi = False, hid = 3, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  18
best_loss = [320.35812

Early stopping at best epoch:  5
best_loss = [571.65216, 543.21857, 527.7856]
time:  2.376997470855713



rnn = LSTM, bi = False, hid = 3, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  143
best_loss = [314.3233, 236.00404, 226.44153]
time:  26.72297477722168



rnn = LSTM, bi = False, hid = 3, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  31
best_loss = [301.0595, 200.58647, 228.69719]
time:  6.6269941329956055



rnn = LSTM, bi = False, hid = 3, num_heads = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  36
best_loss = [248.76562, 151.794, 130.7878]
time:  7.669992685317993



rnn = LSTM, bi = False, hid = 3, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  29
best_loss = [172.72946, 121.1103, 108.07069]
time:  6.261993885040283



rnn = LSTM, bi = False, hid = 3, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  10
best_loss = [242.68784, 144.07541, 103.9124]
time:  3.2739968299865723




Early stopping at best epoch:  29
best_loss = [186.92897, 108.085686, 100.70757]
time:  5.909994125366211



rnn = LSTM, bi = False, hid = 5, num_heads = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  10
best_loss = [204.26178, 109.41444, 147.67973]
time:  3.107996940612793



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = LSTM, bi = False, hid = 5, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  107
best_loss = [361.78275, 302.3447, 287.10074]
time:  17.952982902526855



rnn = LSTM, bi = False, hid = 5, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  159
best_loss = [287.57245, 246.30688, 232.69582]
time:  28.231982231140137



rnn = LSTM, bi = False, hid = 5, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  36
best_loss = [222.5416, 141.81586, 145.81146]
time:  7.4719932079315186



rnn = LSTM, bi = False, hid = 5, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  36
bes

Early stopping at best epoch:  145
best_loss = [276.3319, 207.57774, 194.91826]
time:  27.403144121170044



rnn = LSTM, bi = False, hid = 5, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  29
best_loss = [239.8282, 139.38385, 122.960464]
time:  6.827991962432861



rnn = LSTM, bi = False, hid = 5, num_heads = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  36
best_loss = [234.07965, 150.04163, 129.97635]
time:  8.82599139213562



rnn = LSTM, bi = False, hid = 5, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  19
best_loss = [175.086, 129.55356, 106.367905]
time:  5.437994956970215



rnn = LSTM, bi = False, hid = 5, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  20
best_loss = [168.59055, 152.25494, 129.83133]
time:  5.7019946575164795



rnn = LSTM, bi = False, hid = 5, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  98
best_loss = [321.20602, 248.96005, 214.49608]
time:  17.97898292541

Early stopping at best epoch:  20
best_loss = [184.47452, 134.32292, 104.49716]
time:  5.630500078201294



Pooling: 2 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid = 7, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [572.2348, 543.8474, 528.3646]
time:  2.6989963054656982



rnn = LSTM, bi = False, hid = 7, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  29
best_loss = [562.95966, 523.82416, 512.7656]
time:  6.59929347038269



rnn = LSTM, bi = False, hid = 7, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  43
best_loss = [277.4328, 158.98706, 137.1759]
time:  9.29499077796936



rnn = LSTM, bi = False, hid = 7, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  31
best_loss = [280.45102, 145.7988, 181.99208]
time:  7.136993646621704



rnn = LSTM, bi = False, hid = 7, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  20
best_loss = [235

...

Early stopping at best epoch:  754
best_loss = [183.83418, 79.76704, 129.8082]
time:  187.63582158088684



rnn = RNN, bi = False, hid = 3, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  290
best_loss = [120.68118, 88.61675, 90.564415]
time:  118.23988747596741



rnn = RNN, bi = False, hid = 3, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  258
best_loss = [80.2008, 88.06102, 88.61182]
time:  113.44989228248596



rnn = RNN, bi = False, hid = 3, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  105
best_loss = [129.68939, 114.26792, 96.37599]
time:  90.97891306877136



rnn = RNN, bi = False, hid = 3, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  58
best_loss = [191.11783, 101.88139, 127.602684]
time:  84.17292022705078



rnn = RNN, bi = False, hid = 3, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1109
best_loss = [202.81036, 108.92737, 104.64023]
time:  239.4617722034454

Early stopping at best epoch:  165
best_loss = [113.41449, 93.43854, 117.48417]
time:  106.16589903831482



rnn = RNN, bi = False, hid = 3, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  319
best_loss = [56.71138, 104.37989, 100.10652]
time:  131.8968744277954



rnn = RNN, bi = False, hid = 3, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  729
best_loss = [236.20325, 119.76214, 139.0383]
time:  197.41881227493286



rnn = RNN, bi = False, hid = 3, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  411
best_loss = [238.92122, 116.22309, 153.26016]
time:  147.80585932731628



rnn = RNN, bi = False, hid = 3, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  1063
best_loss = [54.257553, 93.079605, 147.82721]
time:  252.10076022148132



rnn = RNN, bi = False, hid = 3, num_heads = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  289
best_loss = [102.12984, 85.86345, 97.962654]
time:  127.263879

Early stopping at best epoch:  411
best_loss = [219.1441, 121.82747, 116.791885]
time:  139.6018671989441



rnn = RNN, bi = False, hid = 5, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  1400
best_loss = [70.72206, 74.73396, 91.032326]
time:  287.7257261276245



rnn = RNN, bi = False, hid = 5, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  106
best_loss = [144.15997, 115.24255, 104.98652]
time:  92.7249116897583



rnn = RNN, bi = False, hid = 5, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  993
best_loss = [68.10218, 85.72584, 108.09999]
time:  225.88578510284424



rnn = RNN, bi = False, hid = 5, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  362
best_loss = [91.527145, 105.75127, 81.22145]
time:  133.12387323379517



rnn = RNN, bi = False, hid = 5, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1986
best_loss = [573.9347, 526.6565, 521.8309]
time:  378.19264030456543

Early stopping at best epoch:  92
best_loss = [103.758, 79.18819, 97.802635]
time:  85.43691873550415



rnn = RNN, bi = False, hid = 7, num_heads = 5, num_fil = 5, lr = 0.01
Ends at final epoch 3000
Best epoch: 2955
best_loss = [288.5362, 200.05829, 168.67476]
time:  454.3595678806305



rnn = RNN, bi = False, hid = 7, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1481
best_loss = [173.82294, 141.66023, 123.2186]
time:  292.8547213077545



rnn = RNN, bi = False, hid = 7, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  561
best_loss = [187.08734, 85.60301, 155.01642]
time:  162.46184539794922



rnn = RNN, bi = False, hid = 7, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  392
best_loss = [81.09912, 76.965256, 140.9267]
time:  129.16487741470337



rnn = RNN, bi = False, hid = 7, num_heads = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  383
best_loss = [63.48165, 58.69009, 72.18623]
time:  130.585875749

Early stopping at best epoch:  1266
best_loss = [121.095436, 91.320305, 106.3451]
time:  292.3048403263092



rnn = RNN, bi = False, hid = 7, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  299
best_loss = [112.01422, 98.1642, 94.288124]
time:  140.7783932685852



rnn = RNN, bi = False, hid = 7, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  318
best_loss = [113.42623, 79.23381, 114.97134]
time:  138.25112128257751



rnn = RNN, bi = False, hid = 7, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  998
best_loss = [118.25957, 89.92628, 83.87284]
time:  256.3468863964081



rnn = RNN, bi = False, hid = 7, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  403
best_loss = [118.2687, 107.67723, 133.45493]
time:  160.1858730316162



rnn = RNN, bi = False, hid = 7, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  700
best_loss = [239.33032, 131.52016, 117.993866]
time:  217.636740684509

In [ ]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_CA_1DCNN'
rnns = ['BiLSTM', 'BiGRU', 'BiRNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

import time

for [n_ep, patience] in ep_pats:
    print(f"epoch = {n_ep}, patience = {patience}")
    trdl = globals()[f'reg_train_dataloader_{num_cycles}']
    vdl = globals()[f'reg_val_dataloader_{num_cycles}']
    tedl = globals()[f'reg_test_dataloader_{num_cycles}']
    for rnn1, hid1 in itertools.product(rnns, hids):
        bi1 = False
        
        nh2 = 1
        if rnn1.startswith('Bi'):
            bi1 = True
        for pool2, npool2 in itertools.product(pools, npools):
            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
            for fil2, lr in itertools.product(num_fils, lrs):
                start = time.time()
                print(f"rnn = {rnn1}, bi = {bi1}, hid = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                
                # Construct CNN
                model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                # Train
                model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                best_model, best_loss, history, ca = model_train.TrainModel()

                print('best_loss = '+ str(best_loss))

                history_add, state_dict_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                            rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)
                
                # saving best_epoch, loss history, ta score
                save_data(history, history_add)
                save_data(best_model.state_dict(), state_dict_add)
                save_data(ca, ca_add)

                print("time: ", time.time()-start)
                print("\n\n")

In [ ]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

for [n_ep, patience] in ep_pats:
    for rnn1 in rnns:
        df = pd.DataFrame(columns = ['train rmse', 'val rmse', 'test rmse'])
        bi1 = False
        if rnn1.startswith('Bi'):
            bi1 = True
        for hid1 in hids:
            for pool2, npool2 in itertools.product(pools, npools):
                print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers')
                for fil2, lr in itertools.product(num_fils, lrs):
                    history_add, _, _ = history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr)
                    history = load_data(history_add)
                    print(history.iloc[-1, :])

                    df.loc[f'{rnn1}_{hid1}_{fil2}_{pool2}_{npool2}_{fsize2}_{lr}'] = [history.iloc[-1, 0], history.iloc[-1, 1], history.iloc[-1, 2]]

        df
        df_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname], 
                        filename = f'{modelname}_{rnn1}.csv')
        df.to_csv(df_add)

## Plot train/test results

In [ ]:
n_fils = [4, 8, 16]
fsizes = [3]
lrs = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3]
modelname = 'RNN_TA_1DCNN'
modelname2s = ['RNN', 'LSTM', 'GRU']

for n_cy in regression_cycles:
    for n_fil in n_fils:
        for fsize in fsizes:
            for lr in lrs:
                for modelname2 in modelname2s:
                    save_add = f'split_1_{int(max_time/60)}min_padding_{padding_layer}_layers_nonpadding_{nonpadding_layer}_layers_n_fil_{n_fil}_f_size_{fsize}_lr_1_{int(1/lr)}'
                    
                    epoch_add = find_directory(foldername = [f'{n_cy} cycles', modelname, modelname2, 'best_epoch', f'{save_add}.wb'])
                    history_add = find_directory(foldername = [f'{n_cy} cycles', modelname, modelname2, 'train_history', f'{save_add}.pkl'])
                
                    history = load_data(history_add)
                    best_epoch = load_data(epoch_add)
                
                    fig, ax1 = plt.subplots(figsize = (10,4));
                    plt.rcParams["font.size"] = "12";
                    line1 = ax1.plot(np.sqrt(history['mse_loss']), 'y', label = 'train RMSE');
                    line2 = ax1.plot(np.sqrt(history['valmse_loss']), 'b', label = 'val RMSE');
                    ax1.set_xlabel('Epoch');
                    ax1.set_ylabel('RMSE');
                    ax1.set_title(f'{modelname}, {modelname2}, n_fil: {n_fil}, fsize: {fsize}, lr: {lr}, best_epoch: {best_epoch}')

                    ax2 = ax1.twinx();
                    plt.rcParams["font.size"] = "12";
                    line3 = ax2.plot(history['mape_loss'], 'r', label = 'train MAPE');
                    line4 = ax2.plot(history['valmape_loss'], 'g', label = 'val MAPE');
                    ax2.set_ylabel('MAPE');
                    ax2.set_ylim(0, 1);

                    lines = line1 + line2 + line3 + line4;
                    labels = [l.get_label() for l in lines];
                    ax1.legend(lines, labels, loc = 'best');

                figadd = find_directory(foldername = [f'{n_cy} cycles', modelname, modelname2, 'train_val_errors', f'split_1_{int(max_time/60)}min_{n_fil}_{fsize}_{lr}_{best_epoch}.jpg'])
                plt.savefig(figadd);

In [ ]:
regression_cycles = [45, 65, 85, 100]
n_fils = [4, 8, 16]
# purpose = purposes[1]
psize = 2
# n_fils = [4]
fsizes = [3]
lrs = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]
hidden_size  =3
dsets = ['train', 'val', 'test']

modelname = 'RNN_TA_1DCNN'
modelname2s = ['RNN', 'LSTM', 'GRU']

for n_cy in regression_cycles:
    trdl = globals()[f'reg_train_dataloader_{n_cy}']
    vdl = globals()[f'reg_val_dataloader_{n_cy}']
    tedl = globals()[f'reg_test_dataloader_{n_cy}']
    for modelname2 in modelname2s:
        print(modelname2, ':', n_cy, ' cycles')
        df = pd.DataFrame(columns = ['train mape', 'train rmse', 'val mape', 'val rmse', 'test mape', 'test rmse'])
        for n_fil in n_fils:
            for fsize in fsizes:
                for lr in lrs:
#                     purpose = purposes[1]
                    with torch.no_grad():
                        if torch.cuda.is_available():
                            device = 'cuda'
                        else:
                            device = 'cpu'
                            
                        model = globals()[modelname](rnn_num_layer=1, model = modelname2, input_size = n_var, bidirectional = False, hidden_size = hidden_size, num_filter = n_fil,
                              padding_layer = padding_layer, nonpadding_layer = nonpadding_layer, filter_size = fsize, pooling_size = psize,
                              num_time = max_time, num_cycles = n_cy).to(device)
                        model_add = f'split_1_{int(max_time/60)}min_padding_{padding_layer}_layers_nonpadding_{nonpadding_layer}_layers_n_fil_{n_fil}_f_size_{fsize}_lr_1_{int(1/lr)}_state_dict.pth'
                        state_dict = load_data(os.getcwd() + f'\\{n_cy} cycles\\{modelname}\\{modelname2}\\model\\'+model_add)
                        model.load_state_dict(state_dict)
                        model.eval()

                        model_mape, model_rmse = list(), list()


                        for dset in dsets:
                            if dset == 'train':
                                dataloader = trdl
                            elif dset == 'val':
                                dataloader = vdl
                            else:
                                dataloader = tedl

                            predictions, actuals = list(), list()

                            for i, (test_input, test_target) in enumerate(dataloader):
                                test_input = test_input.float().to(device)
                                test_target = test_target.float().to(device)

                                test_yhat, (ta_weight, ta_bias) = model(test_input)

                                if device != 'cpu':
                                    test_yhat = test_yhat.to('cpu')
                                    test_target = test_target.to('cpu')

                                test_yhat = test_yhat.detach().numpy()
                                predictions.append(test_yhat)
                                test_target = test_target.numpy()
                                actuals.append(test_target)

                            pred_acc, targ_acc = np.array(predictions).flatten(), np.array(actuals).flatten()


                            mse = mean_squared_error(actuals, predictions)
                            mape = mean_absolute_percentage_error(actuals, predictions)

                            model_mape.append(mape)
                            model_rmse.append(np.sqrt(mse))

                        df.loc[f'{n_fil}_{fsize}_{lr}'] = [model_mape[0], model_rmse[0], model_mape[1], model_rmse[1], model_mape[2], model_rmse[2]]


        df
        df.to_csv(os.getcwd()+f'\\{n_cy} cycles\\{modelname}\\{modelname2}\\split_1_{int(max_time/60)}min_{modelname}_{modelname2} padding_{padding_layer}_layers_nonpadding_{nonpadding_layer}_layers errors_{n_cy} cycles.csv')